In [ ]:
!pip install transformers[torch] datasets evaluate

In [ ]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]


{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="./sample_data/test_trainer",
                                  evaluation_strategy="epoch",
                                  num_train_epochs=1)

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
#Chame a computação na métrica para calcular a precisão de suas previsões.
#Antes de passar suas previsões para computação, você precisa convertê-las em logits (lembre-se de tudo 🤗 Os modelos de transformadores retornam logits):
# logits são valores numéricos que representam as previsões não normalizadas do modelo antes da aplicação da função de softmax.
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.390582,0.458000


TrainOutput(global_step=125, training_loss=1.5807200927734375, metrics={'train_runtime': 122.7781, 'train_samples_per_second': 8.145, 'train_steps_per_second': 1.018, 'total_flos': 263118142464000.0, 'train_loss': 1.5807200927734375, 'epoch': 1.0})

In [ ]:
trainer.save_model('./sample_data/test_trainer')

In [ ]:
tokenizer.save_pretrained('./sample_data/test_trainer')

('./sample_data/test_trainer/tokenizer_config.json',
 './sample_data/test_trainer/special_tokens_map.json',
 './sample_data/test_trainer/vocab.txt',
 './sample_data/test_trainer/added_tokens.json',
 './sample_data/test_trainer/tokenizer.json')

In [ ]:
from transformers import pipeline

# Carregue o modelo treinado
# Caminho para o diretório onde o modelo foi salvo
model_path = "./sample_data/test_trainer"

classifier = pipeline(
    task="text-classification",
    model=model_path,
    tokenizer=model_path,
)

# Exemplo de pergunta
question = "My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special! The cashier took my friends order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for  when they did  have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. I\'ve worked at more than one location. I expect bad days, bad moods, and the occasional mistake. But I have yet to have a decent experience at this store. It will remain a place I avoid unless someone in my party needs to avoid illness from low blood sugar. Perhaps I should go back to the racially biased service of Steak n Shake instead!"


# Execute a inferência
answer = classifier(question)

# Imprima a resposta
print(answer)

[{'label': 'LABEL_0', 'score': 0.41321563720703125}]


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Caminho para o diretório onde o modelo foi salvo
model_path = "./sample_data/test_trainer"

# Carregue o modelo e o tokenizador
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=5)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Exemplo de pergunta
question = "My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special! The cashier took my friends order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for  when they did  have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. I\'ve worked at more than one location. I expect bad days, bad moods, and the occasional mistake. But I have yet to have a decent experience at this store. It will remain a place I avoid unless someone in my party needs to avoid illness from low blood sugar. Perhaps I should go back to the racially biased service of Steak n Shake instead!"

# Tokenize a pergunta e o contexto
inputs = tokenizer(question, return_tensors="pt")

# Faça a predição
outputs = model(**inputs)

# Obtenha a classe predita (índice com a maior probabilidade)
predicted_class_index = outputs.logits.argmax().item()

# Imprima a resposta
print("Índice da classe predita:", predicted_class_index)

# Agora, você pode acessar os rótulos das classes (se disponíveis) e imprimir o rótulo associado ao índice predito
class_labels = model.config.id2label
predicted_class_label = class_labels[predicted_class_index]

print("Rótulo da classe predita:", predicted_class_label)


Índice da classe predita: 0
Rótulo da classe predita: LABEL_0
